# Visualising the pulsar data

This notebook visualises the frequency, $v$, vs phase data, $\phi$, recorded from the 42 ft telescope at JBO. 

The dispersion in the data is used to calculate the dispersion mesaure $DM$.

Finally, an optimization program is run to obtain the de-dispersed data.

In [ ]:
# Imports and preamble
from __future__ import (print_function, absolute_import, division)
import os
import scipy.optimize as opt

import numpy as np
import scipy

from astropy import coordinates as coord
from astropy import units as u
from astropy import constants as const
from astropy import time as astrotime

from  matplotlib import pyplot as plt
import matplotlib

from math import pi

%matplotlib inline
font = {'family' : 'Arial',
        'weight' : 'normal',
        'size'   : 20}
matplotlib.rc('font', **font)

pulsar_name = 'B0329+54_w1'

## 1. Data Functions

### 1.1 Reading Data

In [ ]:
# Get the data directory path
current_dir = os.getcwd()
root_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
data_dir = os.path.join(root_dir, "Data")
pulsar_meas = os.path.join(data_dir, "PulsarMeasurements")
results_figs = os.path.join(root_dir, "ResultPictures1")

def read_data(file_name):
    """
    Reads the data from ascii files.
    
    @param file_name :: name of the file
    
    @returns :: three arrays with the channel counts,
                phase_counts and data.
    """
    
    file_name = file_name + '.asc'
    data_file = os.path.join(pulsar_meas, file_name)
    
    chan_count, phase_count, data = \
    np.loadtxt(data_file, usecols=(1,2,3), unpack=True, skiprows=1)
    
    return chan_count, phase_count, data

### 1.2 Organising Data

In [ ]:
def get_numbers(chan_count, phase_count):
    """
    Gets the number of channels and the
    number of phase bins of the data.
    
    @param chan_count :: channel counts as read from 
                         the data file
    @param phase_count :: same for phase
    
    @returns :: number of channels and number of phases
    """
    
    nchan=int(chan_count[-1])+1
    nphase=int(phase_count[-1])+1
    
    return nchan, nphase


def reshape_data(nchan, nphase, data):
    """
    Reshapes the data into 2d structure so it is usable.
    
    @param nchan :: number of frequency channels
    @param nphase :: number of phase bins
    @param data :: intensity data
    
    @returns :: 2d data structure, array of channels and
                array of phases
    """
    
    raw = data.reshape(nchan,nphase)
    iphase = phase_count.reshape(nchan,nphase)[0] 
    ichan = chan_count.reshape(nchan,nphase)[:,0]
    
    return raw, ichan, iphase


def freq_integral(raw):
    """
    Takes the data and integrates it over all
    frequency channels.
    
    @param raw :: 2d data structure
    
    @returns :: data integrated along frequency
    """
    
    integrated = np.sum(raw, axis=0)
    
    return integrated


def convert_axis(ichan, iphase):
    """
    Convert the channels of frequency and the phase
    array into physical things.
    
    @param ichan :: array of channels of data
    @param iphase :: array of phases of data
    
    @returns :: array of frequencies and array of times
    """
    
    freq = 605.125 + ichan * (10/nchan)
    time = iphase*(0.71452/nphase)
    
    return freq, time

### 1.3 Plotting the data

In [ ]:
def density_plot(data, freq, time):
    """
    Function that makes a density plot of some data.
    
    @param data :: data of the density plot
    @param freq :: frequencies (y-axis)
    @param time :: times (x-axis)
    
    @returns :: density plot
    """
    
    dens_plot = plt.figure(figsize=(12,7))
    plt.imshow(data, aspect='auto', origin='lower', 
               extent=(time[0],time[-1],freq[0],freq[-1]))
    plt.title(pulsar_name)
    plt.xlabel("Time (s)",)
    plt.ylabel("Frequency (MHz)")
    plt.savefig(results_figs + os.sep + "DensityPlots" + os.sep +
                pulsar_name + ".png")


def intensity_plot(integrated, time):
    """
    Function that makes an intensity plot of some data.
    
    @param integrated :: y-data of the intensity plot
    @param time :: x-data of the intensity plot
  
    @returns :: intensity plot
    """
    
    intens_plot = plt.figure(figsize=(12,7))
    plt.plot(time, integrated)
    plt.xlabel("Phase (iphase)")
    plt.ylabel("Intensity (Arbitrary)")
    plt.title(pulsar_name)
    plt.savefig(results_figs + os.sep + "IntegratedProfiles" + os.sep +
                pulsar_name + ".png")

### 2. Dispersion Measure Functions

### 2.1 Shifting

In [ ]:
def shift_rows(data_in, nchan, shifts):
    """
    Shifts the rows of the data by amount in variable "shifts".
    
    @param data_in :: data to be shifted
    @param shifts :: amount to be shifted by
    
    @returns :: sifted data
    """
    
    shifted=np.zeros_like(data_in)
    for chan in range(nchan):
        shifted[chan] += np.roll(data_in[chan], -int(shifts[chan]))
    return shifted

def shifting(x, constants):
    """
    Shifts the data by an array times the variable m.
    Tries to dedisperse the data this way.
    
    @param constants :: array of constants i.e. raw_data,
                        freq, nphase
    
    @returns :: negative of the maximum of the integrated frequency
    """
    
    bindelay = (x[0] * 4148.8 * constants[2])/(0.71452 * pow(constants[1], 2))
    dedispersed = shift_rows(constants[0], nchan, bindelay)
    integrated = np.sum(dedispersed, axis=0)
    
    return -np.amax(integrated)


def shifting_results(x, constants):
    """
    Does the same thing as function shifting.
    
    @returns :: the bindelay, dedispersed and integrated
                numpy arrays
    """
    
    bindelay = (x[0] * 4148.8 * constants[2])/(0.71452 * pow(constants[1], 2))
    dedispersed = shift_rows(constants[0], nchan, bindelay)
    integrated = np.sum(dedispersed, axis=0)
    
    return bindelay, dedispersed, integrated

### 2.2 Dispersion Measure Through Optimization

In [ ]:
def find_dispersion(shifting, initial_guess, constants):
    """
    Finds the dispersion measure using minimization.
    
    @param initial_guess :: gets the initial guess of the data
    @param shifting :: the shifting function
    @param constants :: array of constants
    
    @returns :: the bindelay, dedispersed and integrated arrays
    """
    
    res = opt.minimize(shifting, [initial_guess], constants, method="Nelder-Mead")
    
    return res.x[0], res.fun

## 3. Kowalski, Analysis.

In [ ]:
# Get the needed data
chan_count, phase_count, data = read_data(pulsar_name)
nchan, nphase = get_numbers(chan_count, phase_count)
raw, ichan, iphase = reshape_data(nchan, nphase, data)
integrated = freq_integral(raw)
freq, time = convert_axis(ichan, iphase)

In [ ]:
density_plot(raw, freq, time)
intensity_plot(integrated, time)

### 3.1 Generate Data for Soumya

Fitting the below curve and roughly apporximating the DM.

In [ ]:
dms = np.linspace(1,100,1000)
intensity_peaks = []
for i, guess in enumerate(dms):
    intensity_peaks.append(-shifting([guess], [raw, freq, nphase, nchan]))

data = np.array([dms, intensity_peaks])
data = np.transpose(data)

np.savetxt(data_dir + os.sep + 'SoumyaData' + os.sep + \
           pulsar_name + '.txt', data, fmt="%.7f")

plt.figure(figsize=(12,7))
plt.plot(data[:,0], data[:,1])
plt.savefig(results_figs + os.sep + "DMCurves" + os.sep +
            pulsar_name + ".png")
plt.show()

### 3.2 Histogram Method

This uses optimization to find dm. Needs fine tuning from the user.

In [ ]:
# Plot 2D histogram to see where the DM
# value would be
n = 1000
tries = np.linspace(20,40,n)
data = []
for i, init_guess in enumerate(tries):
    disp_measure, function_val = \
    find_dispersion(shifting, init_guess, [raw, freq, nphase, nchan])
    data.append([init_guess, disp_measure, -function_val])

# Export the data for generating gifs in mathematica
data = np.asarray(data)
np.savetxt(data_dir + os.sep + 'Histograms' + os.sep + pulsar_name + '.txt', data)

In [ ]:
# Plot the thing
plt.figure(figsize=(12,7))
plt.hist2d(data[:,1], data[:,2], 20, range = [[20,40], [1.40, 1.50]], cmap = 'jet')
plt.colorbar()
plt.show()

#### 3.2.2 Time Delay vs Frequency

In [ ]:
# Save the data to text file for processing in mathematica
freq_delay_dir = os.path.join(data_dir, "FreqDelay")
np.savetxt(freq_delay_dir + os.sep + pulsar_name + "_raw" + ".txt", raw)
np.savetxt(freq_delay_dir + os.sep + pulsar_name + "_time" + ".txt", time)
np.savetxt(freq_delay_dir + os.sep + pulsar_name + "_freq" + ".txt", freq)

### 3.3 Monte Carlo Method

## 4. Noise Estimation

In [ ]:
# Plots the noise in the date
plt.figure(figsize=(10,5))
plt.plot(time,integrated, zorder=1)
plt.hlines(y=0,xmin=0, xmax=0.71452, colors='r', linestyles='solid', zorder=2)

plt.xlabel("Period (seconds)")
plt.ylabel("Intensity (Arbitrary)")
plt.title("Noise in the Integrated profile")
axes = plt.gca()
# If noise is larger, limits need to be changes to estimate upperbound
axes.set_ylim([-0.1,0.1])
plt.show()

# calculates the noise in the integrated array + estimates the uncertainty
Noise = integrated[np.where(integrated < 0.05)]
ErrNoise = np.std(Noise)
print("Statistical Uncertainty due to Noise = {}".format(ErrNoise))